In [2]:
# Import libraries
from functools import reduce
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt, exp
from pandas_datareader.data import DataReader
import requests_cache
%matplotlib inline

In [17]:
# Download data from pandas datareader -> AAPL.O, IBM.N, GOOG.O, BP.N, XOM.N, COST.O, GS.N
# Start: 01/01/2016
# End: 31/12/2016
# Create a dictionary: Key -> Ticker, Value -> Historical data from Start_Date to End_Date

session = requests_cache.CachedSession(cache_name='cache',backend='sqlite')
session.headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
                   'Accept': 'application/json;charset=utf-8'}

def get_data(tickers, start_date="2016-01-01", end_date="2016-12-31"):
    stocks = dict()
    for ticker in tickers:
        s = DataReader(ticker, 'yahoo', start_date, end_date, session=session)
        s.insert(0, "Ticker", ticker)
        s['Prev Close'] = s['Adj Close'].shift(1)
        s['log_return'] = np.log(s['Adj Close']/s['Prev Close'])
        s['perc_return'] = (s['Adj Close']/s['Prev Close']) - 1
        stocks[ticker] = s
    return stocks

# Initialize the dataset with historical data
stocks = ["AAPL", "IBM", "GOOG", "BP", "XOM", "COST", "GS"]
portfolio = get_data(stocks)

## Question 4a. Using historical daily returns, calculate VaR95% and CVaR95% of portfolio as of 2016/12/31

In [28]:
# Set weights of stocks in portfolio
weights = [0.15, 0.2, 0.2, 0.15,0.1,0.15,0.05]

for stock in stocks:
    portfolio[stock]['Previous Adj Close'] = portfolio[stock]['Adj Close'].shift(1)
portfolio['AAPL'].head(3)

,Ticker,High,Low,Open,Close,Volume,Adj Close,Prev Close,log_return,perc_return,Previous Adj Close
Date,,,,,,,,,,,
2015-12-31,AAPL,26.757500,26.205000,26.752501,26.315001,163649200.0,24.266081,NaN,NaN,NaN,NaN
2016-01-04,AAPL,26.342501,25.500000,25.652500,26.337500,270597600.0,24.286833,24.266081,0.000855,0.000855,24.266081
2016-01-05,AAPL,26.462500,25.602501,26.437500,25.677500,223164000.0,23.678219,24.286833,-0.025379,-0.025059,24.286833


In [30]:
portfolio_df = pd.DataFrame()
portfolio_df.insert(0, "Date", portfolio['AAPL'].index)
portfolio_df = portfolio_df.set_index("Date")